In [1]:
%load_ext autoreload
%autoreload 2
    

# Running MD simulations using nff and ASE

This Jupyter Notebook shows how the `nff` package interfaces with the Atomistic Simulation Environment (ASE). We assume the user went through tutorial `01_training`, so we can load the pretrained models without having to train them again.

As before, importing the dependencies:

In [2]:
import numpy as np
import matplotlib.pyplot as plt

import torch
from ase import Atoms
from ase.md.verlet import VelocityVerlet

from nff.md.nve import Dynamics
from nff.data import Dataset
from nff.train import load_model, evaluate
import nff.utils.constants as const
from ase import units
from nff.io import NeuralFF, AtomsBatch

## Loading the relevant data

We reload the dataset and create a `GraphLoader` as we did last time:

In [3]:
dataset = Dataset.from_file('data/dataset.pth.tar')


### Creating Atoms

As before, we can create an `Atoms` object from any element of the dataset. Let's take the first one, for simplicity:

In [4]:
props = dataset[0].copy()
atoms = AtomsBatch(positions=props['nxyz'][:, 1:], 
                   numbers=props['nxyz'][:, 0], 
                   props=props
                   )


### Creating the ASE calculator

Now we just have to load the ASE calculator from a pretrained model. One way of doing so is through the in-build `from_file` method. You just have to specify the folder where the model was trained and subsequently stored.

In [5]:
nff_ase = NeuralFF.from_file('sandbox/', device=0)

/home/saxelrod/miniconda3/envs/htvs/lib/python3.7/site-packages/torch/serialization.py:658: SourceChangeWarning: source code of class 'nff.nn.models.schnet.SchNet' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/home/saxelrod/miniconda3/envs/htvs/lib/python3.7/site-packages/torch/serialization.py:658: SourceChangeWarning: source code of class 'torch.nn.modules.sparse.Embedding' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/home/saxelrod/miniconda3/envs/htvs/lib/python3.7/site-packages/torch/serialization.py:658: SourceChangeWarning: source code of class 'torch.nn.modules.container.ModuleList' has changed. you can retrieve 

Assigning this calculator to `atoms` is easy:

In [6]:
atoms.set_calculator(nff_ase)

### Configuring the dynamics for the system

In this example, we will run an NVE dynamics simulation. We will use the default parameters there implemented to run a trajectory for an ethanol molecule. The parameters we will specify are the following:

* `T_init`: initial temperature of the simulation
* `time_step`: time step in femtoseconds
* `thermostat`: ASE integrator to use when performing the simulation
* `thermostat_params`: keyword arguments for ase.Integrator class, will be different case-by-case
* `steps`: number of steps to simulate
* `save_frequency`: how often (in steps) save the pose of the molecule in a file
* `nbr_list_update_freq`: how often (in steps) to update the neighbor list (not yet implemented)
* `thermo_filename`: output file for the thermodynamics log
* `traj_filename`: output file for the ASE trajectory file
* `skip`: number of initial frames to skip when recording the trajectory

In [7]:
md_params = {
    'T_init': 450,
    'time_step': 0.5,
#     'thermostat': NoseHoover,   # or Langevin or NPT or NVT or Thermodynamic Integration
#     'thermostat_params': {'timestep': 0.5 * units.fs, "temperature": 120.0 * units.kB,  "ttime": 20.0}
    'thermostat': VelocityVerlet,  
    'thermostat_params': {'timestep': 0.5 * units.fs},
    'steps': 200,
    'save_frequency': 10,
    'nbr_list_update_freq': 3,
    'thermo_filename': 'thermo.log',
    'traj_filename': 'atoms.traj',
    'skip': 0
}

In [8]:
nve = Dynamics(atoms, md_params)
nve.run()

Time[ps]      Etot[eV]     Epot[eV]     Ekin[eV]    T[K]
0.0000           0.4920      -0.1916       0.6836   587.6



/home/saxelrod/miniconda3/envs/htvs/lib/python3.7/site-packages/ase/atoms.py:921: VisibleDeprecationWarning: Use get_global_number_of_atoms() instead
  np.VisibleDeprecationWarning)
/home/saxelrod/Repo/projects/master/NeuralForceField/nff/nn/utils.py:121: UserWarning: This overload of nonzero is deprecated:
	nonzero()
Consider using one of the following signatures instead:
	nonzero(*, bool as_tuple) (Triggered internally at  /opt/conda/conda-bld/pytorch_1603729138878/work/torch/csrc/utils/python_arg_parser.cpp:882.)
  nbr_list = mask.nonzero()


0.0050           0.4901      -0.1788       0.6688   574.9

0.0100           0.4974       0.0966       0.4008   344.5

0.0150           0.4990      -0.0012       0.5002   430.0

0.0200           0.4973      -0.0424       0.5397   463.9

0.0250           0.4940      -0.0835       0.5776   496.5

0.0300           0.4943      -0.0843       0.5786   497.4

0.0350           0.4891      -0.3113       0.8004   688.0

0.0400           0.4892      -0.1262       0.6153   528.9

0.0450           0.4943      -0.0209       0.5153   442.9

0.0500           0.4930      -0.0722       0.5653   485.9

0.0550           0.4974       0.1738       0.3236   278.2

0.0600           0.4982      -0.0124       0.5106   438.9

0.0650           0.4964       0.0187       0.4776   410.6

0.0700           0.4899      -0.1569       0.6467   555.9

0.0750           0.4913      -0.1833       0.6746   579.9

0.0800           0.4940      -0.1534       0.6474   556.5

0.0850           0.4915      -0.1513       0.6428   552.

### Models with directed neighbor lists

The default assumes that you're using SchNet, which uses an undirected neighbor list to save memory. If you're using Painn, DimeNet, or any model with directional information, you will need a directed neighbor list. If you don't specify this then you will get an error. 

While these models automatically make any neighbor list into a directed one when called, the same is not true of periodic boundary conditions. The dataest offsets from the PBC depend on whether the dataset neighbor list is directed or not.

In [9]:
props = dataset[0].copy()
atoms = AtomsBatch(positions=props['nxyz'][:, 1:], 
                   numbers=props['nxyz'][:, 0], 
                   props=props,
                   )
nff_ase = NeuralFF.from_file('sandbox_painn/', device=0)
atoms.set_calculator(nff_ase)

try:
    nve = Dynamics(atoms, md_params)
    nve.run()
except Exception as e:
    print(e)

Time[ps]      Etot[eV]     Epot[eV]     Ekin[eV]    T[K]
Painn needs a directed neighbor list


If you do then you will be fine!

In [10]:
props = dataset[0].copy()
atoms = AtomsBatch(positions=props['nxyz'][:, 1:], 
                   numbers=props['nxyz'][:, 0], 
                   props=props,
                   directed=True)
nff_ase = NeuralFF.from_file('sandbox_painn/', device=0)
atoms.set_calculator(nff_ase)

nve = Dynamics(atoms, md_params)

try:
    nve.run()
except Exception as e:
    print(e)
    import pdb
    pdb.post_mortem()

Time[ps]      Etot[eV]     Epot[eV]     Ekin[eV]    T[K]
0.0000           0.5966      -0.1615       0.7581   651.7

0.0050           0.5971      -0.2297       0.8268   710.7

0.0100           0.5993      -0.0914       0.6907   593.7

0.0150           0.5980      -0.0264       0.6244   536.7

0.0200           0.5991      -0.1291       0.7282   625.9

0.0250           0.5994      -0.0903       0.6897   592.9

0.0300           0.5984      -0.0981       0.6965   598.7

0.0350           0.5987      -0.0813       0.6799   584.5

0.0400           0.5984      -0.1087       0.7072   607.9

0.0450           0.5979      -0.1205       0.7184   617.6

0.0500           0.5986      -0.0450       0.6437   553.3

0.0550           0.5981      -0.2268       0.8249   709.0

0.0600           0.5962      -0.2912       0.8874   762.8

0.0650           0.5986      -0.0123       0.6109   525.2

0.0700           0.5975      -0.1596       0.7570   650.8

0.0750           0.5988      -0.0462       0.6450   554.4


The dynamics conserved the energy. The temperature varied throughout the simulation, as expected.

# Additional properties and options

You can also specify the properties that you want the calculator to predict. The default is `energy` and `forces`, but you can also add `stress`.

However, if you run this for ethanol you will get an error, because it doesn't have any lattice vectors:

In [11]:
nff_ase.properties = ['energy', 'forces', 'stress']
atoms.set_calculator(nff_ase)

try:
    nve = Dynamics(atoms, md_params)
    nve.run()
except Exception as e:
    print(e)

Time[ps]      Etot[eV]     Epot[eV]     Ekin[eV]    T[K]      ---------------------- stress [GPa] -----------------------
You have 0 lattice vectors: volume not defined


If we make an artificial unit cell, it all works fine:

In [12]:
nff_ase.properties = ['energy', 'forces', 'stress']
atoms.set_calculator(nff_ase)
atoms.set_cell(2 * np.identity(3))
nve = Dynamics(atoms, md_params)
nve.run()



Time[ps]      Etot[eV]     Epot[eV]     Ekin[eV]    T[K]      ---------------------- stress [GPa] -----------------------
0.0000           0.4434      -0.0888       0.5322   457.5     15.528     29.909     55.015      0.639    -29.884    -28.244

0.0050           0.4444       0.0253       0.4192   360.3     56.424     31.878    -36.799     34.077     -3.453    -68.015

0.0100           0.4448      -0.0264       0.4712   405.1    -18.224    -11.406     -9.799    -24.066    -14.712     16.169

0.0150           0.4430      -0.1227       0.5657   486.3     21.704     19.513     58.857    -24.071    -26.487     17.447

0.0200           0.4414      -0.0477       0.4891   420.4    -60.570    -47.979    -37.840      3.866     32.923     49.813

0.0250           0.4441      -0.0523       0.4964   426.7    -37.065     65.320     82.556     -7.451     19.073      1.422

0.0300           0.4426      -0.3040       0.7466   641.8    -14.836    -13.776    -71.491     39.054     27.461    -39.897

0.0

Since `stress` is an implemented property, we can get the stress on the atoms:

In [13]:
atoms.get_stress()

array([-0.64447266, -0.11506233, -0.39281893,  0.06315393,  0.099292  ,
       -0.2362834 ], dtype=float32)

You can additionally request larger offsets for periodic structures than the default. To do so, use `requires_large_offsets=True` when making the atoms:

In [14]:
lattice = [[1.155363155, 0.0, 0.0], 
           [0.0, 1.155363155, 0.0], 
           [0.0, 0.0, 1.07085842]]

# default of `requires_large_offsets` is False
old_atoms = AtomsBatch(positions=props['nxyz'][:, 1:], 
                       numbers=props['nxyz'][:, 0], 
                       props=props,
                       cell=lattice,
                       pbc=True
                       )

new_atoms = AtomsBatch(positions=props['nxyz'][:, 1:], 
                       numbers=props['nxyz'][:, 0], 
                       props=props,
                       requires_large_offsets=True,
                       cell=lattice,
                       pbc=True
                       )

If we compare the offsets in the new and old `atoms` objects, we see that there are differences:

In [15]:
old_nbrs, old_offsets = old_atoms.update_nbr_list()
new_nbrs, new_offsets = new_atoms.update_nbr_list()

print((old_offsets.to_dense() == new_offsets.to_dense()).all().item())
print(old_offsets)
print(new_offsets)


False
tensor(indices=tensor([[ 0,  0,  1,  1,  2,  2,  3,  4,  5,  5,  5,  6,  6,  7,
                         7,  8,  9,  9, 10, 10, 10, 11, 11, 12, 12, 13, 13, 14,
                        15, 15, 16, 16, 16, 17, 17, 18, 18, 18, 19, 19, 19, 20,
                        21, 22, 22, 23, 23, 24, 24, 24, 25, 25, 26, 26, 27, 27,
                        27, 28, 28, 29, 29, 29, 30, 30, 30, 31, 31, 31, 32, 33,
                        34, 34, 35, 35, 35],
                       [ 0,  1,  0,  1,  1,  2,  2,  0,  0,  1,  2,  0,  1,  0,
                         2,  0,  0,  1,  0,  1,  2,  0,  1,  1,  2,  1,  2,  0,
                         0,  1,  0,  1,  2,  0,  1,  0,  1,  2,  0,  1,  2,  2,
                         2,  0,  1,  0,  1,  0,  1,  2,  0,  1,  0,  2,  0,  1,
                         2,  0,  1,  0,  1,  2,  0,  1,  2,  0,  1,  2,  0,  2,
                         0,  1,  0,  1,  2]]),
       values=tensor([ 1.1554,  1.1554, -1.1554,  1.1554, -1.1554,  1.0709,
                      -1.0

## Visualizing the trajectory

To visualize the trajectory in this Jupyter Notebook, you will have to install the package [nglview](https://github.com/arose/nglview).

In [16]:
import nglview as nv
from ase.io import Trajectory

Displaying the trajectory:

In [17]:
%matplotlib notebook

traj = Trajectory('atoms.traj')
nv.show_asetraj(traj)

NGLWidget(max_frame=19)

Looks like the atoms are still together. Visual inspection says that the trajectory is reasonable. Yay for `nff`!